In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# 작업한 파일 로컬로 다운로드 함수
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

# 1. 기준 통일

In [227]:
df5 = pd.read_csv('./data/5.112신고.csv')
df5.head()

,date,jur_stn,report_sx,inc_info
0,20180603.0,서울서부,남성,분실습득
1,20180603.0,서울송파,여성,폭력
2,20180603.0,서울강서,남성,소음
3,20180603.0,서울영등포,여성,실종(실종아동 등)
4,20180603.0,서울강서,남성,상담문의


In [251]:
df5[df5['jur_stn'].str.contains('세종')][33350:33400]

,date,jur_stn,report_sx,inc_info
7538387,20190625.0,세종,남성,시비
7538388,20190625.0,세종,여성,주취자
7538389,20190625.0,세종,NaN,내용확인불가
7538390,20190625.0,세종,불상,내용확인불가
7538391,20190625.0,세종,남성,행패소란
7538392,20190625.0,세종,남성,상담문의
7538393,20190625.0,세종,남성,교통사고
7538394,20190625.0,세종,남성,교통사고
7538395,20190625.0,세종,남성,교통불편
7538396,20190625.0,세종,남성,분실습득


In [82]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9229111 entries, 0 to 9229110
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   date       float64
 1   jur_stn    object 
 2   report_sx  object 
 3   inc_info   object 
dtypes: float64(1), object(3)
memory usage: 281.7+ MB


In [83]:
df5.isnull().sum()

date              0
jur_stn           0
report_sx      2190
inc_info     145216
dtype: int64

In [84]:
# 경찰 리스트 매칭

In [85]:
# date 전처리
df5['date']=df5['date'].astype(int)
df5['year'] = df5['date'].apply(lambda x: (str(x)[:4]))
df5['month'] = df5['date'].apply(lambda x: (str(x)[4:6]))
df5['year_half'] = df5['month'].apply(lambda x: '(하반기)' if int(x)>6 else '(상반기)')

# 기준에 맞춘 ym 컬럼 생성
df5['ym'] = df5['year'] + df5['year_half']
df5.head()

,date,jur_stn,report_sx,inc_info,year,month,year_half,ym
0,20180603,서울서부,남성,분실습득,2018,06,(상반기),2018(상반기)
1,20180603,서울송파,여성,폭력,2018,06,(상반기),2018(상반기)
2,20180603,서울강서,남성,소음,2018,06,(상반기),2018(상반기)
3,20180603,서울영등포,여성,실종(실종아동 등),2018,06,(상반기),2018(상반기)
4,20180603,서울강서,남성,상담문의,2018,06,(상반기),2018(상반기)


In [86]:
# 경찰서명 통일
## 5번 파일 경찰서 가지수는 기준과 동일
print(df5['jur_stn'].unique())
print(len(df5['jur_stn'].unique()))

['서울서부' '서울송파' '서울강서' '서울영등포' '서울양천' '서울강동' '서울관악' '서울중랑' '서울금천' '서울용산'
 '서울광진' '서울방배' '서울성동' '서울수서' '서울동대문' '서울노원' '서울마포' '서울서대문' '서울강남' '서울종로'
 '서울도봉' '서울강북' '서울동작' '서울서초' '서울구로' '서울성북' '서울중부' '서울은평' '서울종암' '서울혜화'
 '서울남대문' '세종' '수원중부' '수원서부' '수원남부' '진주' '창원중부' '창원서부' '진해' '마산동부' '마산중부']
41


In [87]:
# 경찰서명을 살펴보니 뒤에 경찰서만 붙이면 같음
df5['jur_stn_pro'] = df5['jur_stn'].apply(lambda x: (x+'경찰서'))
df5.head()

,date,jur_stn,report_sx,inc_info,year,month,year_half,ym,jur_stn_pro
0,20180603,서울서부,남성,분실습득,2018,06,(상반기),2018(상반기),서울서부경찰서
1,20180603,서울송파,여성,폭력,2018,06,(상반기),2018(상반기),서울송파경찰서
2,20180603,서울강서,남성,소음,2018,06,(상반기),2018(상반기),서울강서경찰서
3,20180603,서울영등포,여성,실종(실종아동 등),2018,06,(상반기),2018(상반기),서울영등포경찰서
4,20180603,서울강서,남성,상담문의,2018,06,(상반기),2018(상반기),서울강서경찰서


In [88]:
df1= gpd.read_file('./data/1.5개시도_경찰서_관할경계.geojson')
df1.head()

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."
3,창원중부경찰서,경남청,"MULTIPOLYGON (((128.60966 35.15093, 128.60956 ..."
4,마산동부경찰서,경남청,"MULTIPOLYGON (((128.62696 35.21714, 128.62695 ..."


In [89]:
# 1번 파일(기준) 경찰서 명과 5번파일 경찰서명 통일 여부 검증
## 1번 파일 경찰서명 리스트
pol_list =[]
for i in range(len(df1)):
    pol_list.append(df1.NAME[i])

## 5번 파일 경찰서명 중복제거    
df_t =df5['jur_stn_pro']
df_t.drop_duplicates(inplace=True)    

pol_list2 =[]
for i in df_t:
    pol_list2.append(i)

In [90]:
test =[]
for i in pol_list:
    if i not in pol_list2:
        test.append(i)
test
# 1번 파일의 경찰서명과 전처리한 5번 파일의 경찰서명이 동일함을 확인

[]

In [91]:
# 불필요한 컬럼 제외하고 내부 정보를 가져올 수 있는 데이터프레임으로 변환
df5 = df5.drop(columns=['date','jur_stn','year','year_half'])
df5 = df5[['jur_stn_pro','ym','report_sx','inc_info','month']]
df5.rename(columns={'jur_stn_pro' : 'jur_stn', 'ym' : 'year'}, inplace=True)
df5

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,06
1,서울송파경찰서,2018(상반기),여성,폭력,06
2,서울강서경찰서,2018(상반기),남성,소음,06
3,서울영등포경찰서,2018(상반기),여성,실종(실종아동 등),06
4,서울강서경찰서,2018(상반기),남성,상담문의,06
...,...,...,...,...,...
9229106,마산동부경찰서,2021(상반기),남성,교통사고,05
9229107,마산중부경찰서,2021(상반기),남성,가정폭력,05
9229108,마산중부경찰서,2021(상반기),불상,화재,05
9229109,마산동부경찰서,2021(상반기),남성,기타형사범,05


In [95]:
# 추후 작업을 위해 데이터셋 분리
df5.to_csv('./5.112신고_processed.csv', index=False)

In [93]:
# 데이터 백업을 위한 로컬 저장소 활용
csv_download_link(df5,'5.112신고_processed.csv')

/opt/app-root/src/5.112신고_processed.csv

Press enter to delete the file after you have downloaded it.


# 2. 결측치 작업
- 성별 : 불상으로처리
- 신고누락 : drop처리
- 2021년 데이터 : 일단 제외

In [325]:
# 기준 통일 시킨 데이터 불러오기
df5 = pd.read_csv('./5.112신고_processed.csv')
df5.head()

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,6
1,서울송파경찰서,2018(상반기),여성,폭력,6
2,서울강서경찰서,2018(상반기),남성,소음,6
3,서울영등포경찰서,2018(상반기),여성,실종(실종아동 등),6
4,서울강서경찰서,2018(상반기),남성,상담문의,6


In [326]:
df5.isnull().sum()

jur_stn           0
year              0
report_sx      2190
inc_info     145216
month             0
dtype: int64

In [327]:
df5.report_sx.unique()

array(['남성', '여성', '불상', nan], dtype=object)

In [328]:
# 성별 결측치의 경우 불상으로 판단해도 무방하다
df5.report_sx.fillna('불상', inplace = True)
df5.isnull().sum()

jur_stn           0
year              0
report_sx         0
inc_info     145216
month             0
dtype: int64

In [329]:
df5.inc_info.unique()

array(['분실습득', '폭력', '소음', '실종(실종아동 등)', '상담문의', '보호조치', '교통사고', '기타형사범',
       '절도', '교통불편', '행패소란', '위험방지', '내용확인불가', '변사자', '기타_타기관', '구조요청',
       '주취자', '재물손괴', '청소년비행', '교통위반', '도박', '비상벨', '서비스요청', '가정폭력', '시비',
       '사기', '기타경범', '성폭력', '자살', '무전취식승차', '가출 등', '화재', '수배불심자',
       '데이트폭력', '풍속영업', '납치감금', '주거침입', '스토킹', '치기', '협박', '노점상',
       '아동학대(가정내)', '아동학대(기타)', '살인', '인피도주', '경비업체요청', 'FTX', '강도', '공갈',
       '사망.대형사고', '청탁금지법', '학교폭력', '위험동물', '재해재난', nan], dtype=object)

In [330]:
# 약 92만개 데이터 충 신고내용이 확인 불가능한 경우는 1.4만건이다
# 해당 내용은 결측 보완이 불가능할 뿐더러 잘 못 신고가 간 경우로 판단하여 결측치를 제거해준다.
df5.dropna(inplace = True)
df5.index = range(len(df5))
df5.isnull().sum()

jur_stn      0
year         0
report_sx    0
inc_info     0
month        0
dtype: int64

In [332]:
# 2021년 데이터 삭제
df5 = df5[(df5['year']!='2021(상반기)') & (df5['year']!='2021(하반기)')]
df5.head()

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,6
1,서울송파경찰서,2018(상반기),여성,폭력,6
2,서울강서경찰서,2018(상반기),남성,소음,6
3,서울영등포경찰서,2018(상반기),여성,실종(실종아동 등),6
4,서울강서경찰서,2018(상반기),남성,상담문의,6


In [333]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7987031 entries, 0 to 9044236
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   jur_stn    object
 1   year       object
 2   report_sx  object
 3   inc_info   object
 4   month      int64 
dtypes: int64(1), object(4)
memory usage: 365.6+ MB


# 3. 데이터 특성 파악 및 취합
- 2018(상반기) 누락 데이터가 있어서 비율 조정
- 전체 건수, 5대범죄 건수별 정렬

In [337]:
# 2018 상반기는 6월만 있음 -> 상반기 비율을 확인해서 추후 작업에 비율 조정
df5['year'].value_counts()

2019(하반기)    1706381
2018(하반기)    1566665
2019(상반기)    1535951
2020(하반기)    1476794
2020(상반기)    1422313
2018(상반기)     278927
Name: year, dtype: int64

In [338]:
# 19년 상반기, 20년 상반기의 6월 비중 확인
june19 = df5[(df5['year'] == '2019(상반기)') &(df5['month']==6)]['month'].value_counts().sum()
total19 = df5[df5['year'] == '2019(상반기)']['month'].value_counts().sum()

june20 = df5[(df5['year'] == '2020(상반기)') &(df5['month']==6)]['month'].value_counts().sum()
total20 = df5[df5['year'] == '2020(상반기)']['month'].value_counts().sum()

print(june19/total19)
print(june20/total20)

# => 매년 6월에 약 19~20% 신고건수 발생 -> 추후 18년도 상반기 작업에 수치 X 5 반영

0.1957015555834789
0.1886539741955533


In [339]:
# 비중 확인 완료했으니 month 컬럼 제거
# df5.drop(columns=['month'], inplace=True)
df5.head()

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,6
1,서울송파경찰서,2018(상반기),여성,폭력,6
2,서울강서경찰서,2018(상반기),남성,소음,6
3,서울영등포경찰서,2018(상반기),여성,실종(실종아동 등),6
4,서울강서경찰서,2018(상반기),남성,상담문의,6


## 3-1.성별 신고수 작업

In [340]:
# 성별 신고수 카운트
import copy
police_call = df5.copy()

# police_call을 생성해서 전체 신고수 확인
police_call = police_call.groupby(['jur_stn','year']).count()
police_call.reset_index(inplace=True)
police_call.rename(columns = {'report_sx' : 'report_sx_cnt'},inplace=True)
police_call.drop(columns=['inc_info','month'],inplace=True)

# 2018(상반기) 비중 5배 작업
for i in range(len(police_call)):
    if police_call['year'][i] == '2018(상반기)':
        police_call['report_sx_cnt'][i] *=5
police_call.head()

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,jur_stn,year,report_sx_cnt
0,마산동부경찰서,2018(상반기),17755
1,마산동부경찰서,2018(하반기),20677
2,마산동부경찰서,2019(상반기),23935
3,마산동부경찰서,2019(하반기),26014
4,마산동부경찰서,2020(상반기),22971


In [341]:
police_sx = df5.copy()
police_sx.drop(columns=['inc_info'],inplace=True)
police_sx = police_sx.groupby(['jur_stn','year','report_sx']).count()
police_sx.reset_index(inplace=True)
police_sx.rename(columns={'month' : 'report_sx_cnt'},inplace=True)

# 얘도 2018(상반기) X 5 해줌
for i in range(len(police_sx)):
    if police_sx['year'][i] == '2018(상반기)':
        police_sx['report_sx_cnt'][i] *=5
police_sx.head()

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,jur_stn,year,report_sx,report_sx_cnt
0,마산동부경찰서,2018(상반기),남성,10485
1,마산동부경찰서,2018(상반기),불상,1785
2,마산동부경찰서,2018(상반기),여성,5485
3,마산동부경찰서,2018(하반기),남성,12327
4,마산동부경찰서,2018(하반기),불상,2011


In [342]:
# 남성/여성/불상 숫자 넣을 temp dataframe 생성
temp_data = police_call.drop_duplicates(subset=['jur_stn','year'])[['jur_stn','year']]

for i in police_sx['report_sx'].unique():
    temp_data['report_sx_'+i] = 0

for i in range(len(police_sx)) :
    temp_name = police_sx['report_sx'].iloc[i]
    temp = temp_data[(temp_data['jur_stn']==police_sx['jur_stn'].iloc[i])&(temp_data['year']==police_sx['year'].iloc[i])]
    temp[temp_name] = police_sx['report_sx_cnt'].iloc[i]
    temp_data['report_sx_'+temp_name].iloc[temp.index] = police_sx['report_sx_cnt'].iloc[i]
police_sx = temp_data
police_sx.head(3)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성
0,마산동부경찰서,2018(상반기),10485,1785,5485
1,마산동부경찰서,2018(하반기),12327,2011,6339
2,마산동부경찰서,2019(상반기),14080,2610,7245


In [343]:
# 성별 total 데이터셋 합쳐주기
police_sx['report_sx_total'] = police_call['report_sx_cnt']
police_sx

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total
0,마산동부경찰서,2018(상반기),10485,1785,5485,17755
1,마산동부경찰서,2018(하반기),12327,2011,6339,20677
2,마산동부경찰서,2019(상반기),14080,2610,7245,23935
3,마산동부경찰서,2019(하반기),15177,3190,7647,26014
4,마산동부경찰서,2020(상반기),12967,3348,6656,22971
...,...,...,...,...,...,...
238,창원중부경찰서,2018(하반기),13407,2772,6438,22617
239,창원중부경찰서,2019(상반기),15598,3216,7456,26270
240,창원중부경찰서,2019(하반기),16940,3513,8094,28547
241,창원중부경찰서,2020(상반기),14667,3315,7327,25309


In [344]:
# 세종은 19년 하반기~20년 상/하반기 데이터가 없음
police_sx[police_sx['jur_stn']=='세종경찰서']

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total
198,세종경찰서,2018(상반기),4325,1415,1805,7545
199,세종경찰서,2018(하반기),6050,1768,2775,10593
200,세종경찰서,2019(상반기),11388,3862,6028,21278


In [345]:
# base_df 호출 및 없는 row 삭제
base_df=pd.read_csv('./base_df.csv')
base_df=base_df[(base_df['year']!='2016(이전)')&(base_df['year']!='2017(상반기)')&(base_df['year']!='2017(하반기)')]
base_df.index=range(len(base_df))
base_df

,jur_stn,year
0,서울중부경찰서,2018(상반기)
1,서울종로경찰서,2018(상반기)
2,서울남대문경찰서,2018(상반기)
3,서울서대문경찰서,2018(상반기)
4,서울혜화경찰서,2018(상반기)
...,...,...
241,창원서부경찰서,2020(하반기)
242,마산중부경찰서,2020(하반기)
243,마산동부경찰서,2020(하반기)
244,진주경찰서,2020(하반기)


In [346]:
# base_df merge해서 0값으로 가져오기
police_sx = pd.merge(base_df,police_sx,how='outer',on=('jur_stn','year'))
police_sx.fillna(0.0,inplace=True)
police_sx

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total
0,서울중부경찰서,2018(상반기),13730.0,3430.0,6310.0,23470.0
1,서울종로경찰서,2018(상반기),8215.0,2260.0,4315.0,14790.0
2,서울남대문경찰서,2018(상반기),9315.0,1430.0,3380.0,14125.0
3,서울서대문경찰서,2018(상반기),18685.0,9855.0,10670.0,39210.0
4,서울혜화경찰서,2018(상반기),11340.0,2370.0,5530.0,19240.0
...,...,...,...,...,...,...
241,창원서부경찰서,2020(하반기),11915.0,2751.0,6897.0,21563.0
242,마산중부경찰서,2020(하반기),9913.0,2094.0,5405.0,17412.0
243,마산동부경찰서,2020(하반기),13687.0,3432.0,6906.0,24025.0
244,진주경찰서,2020(하반기),18826.0,4426.0,11375.0,34627.0


In [347]:
police_sx.isnull().sum()

jur_stn            0
year               0
report_sx_남성       0
report_sx_불상       0
report_sx_여성       0
report_sx_total    0
dtype: int64

In [348]:
police_sx.jur_stn.value_counts()

서울중랑경찰서     6
서울동대문경찰서    6
서울용산경찰서     6
창원중부경찰서     6
서울동작경찰서     6
서울양천경찰서     6
서울노원경찰서     6
서울서대문경찰서    6
서울강서경찰서     6
서울종암경찰서     6
서울종로경찰서     6
수원중부경찰서     6
서울성동경찰서     6
서울금천경찰서     6
진해경찰서       6
서울혜화경찰서     6
수원서부경찰서     6
진주경찰서       6
서울성북경찰서     6
서울송파경찰서     6
창원서부경찰서     6
수원남부경찰서     6
서울구로경찰서     6
서울서초경찰서     6
서울강남경찰서     6
서울마포경찰서     6
서울서부경찰서     6
서울강동경찰서     6
서울영등포경찰서    6
서울방배경찰서     6
서울관악경찰서     6
서울수서경찰서     6
세종경찰서       6
서울남대문경찰서    6
마산중부경찰서     6
마산동부경찰서     6
서울광진경찰서     6
서울중부경찰서     6
서울도봉경찰서     6
서울강북경찰서     6
서울은평경찰서     6
Name: jur_stn, dtype: int64

In [349]:
# 세종은 일단 빈값으로 두자
police_sx[police_sx['jur_stn'].str.contains('세종')]

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total
34,세종경찰서,2018(상반기),4325.0,1415.0,1805.0,7545.0
75,세종경찰서,2018(하반기),6050.0,1768.0,2775.0,10593.0
116,세종경찰서,2019(상반기),11388.0,3862.0,6028.0,21278.0
154,세종경찰서,2019(하반기),0.0,0.0,0.0,0.0
198,세종경찰서,2020(상반기),0.0,0.0,0.0,0.0
239,세종경찰서,2020(하반기),0.0,0.0,0.0,0.0


## 3-2. 폭력 신고수 작업

In [350]:
# 폭력 관련 / 강도,살인 관련 / 교통사고 관련 이슈 취합
police_vio = df5.copy()
police_vio.head(3)

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,6
1,서울송파경찰서,2018(상반기),여성,폭력,6
2,서울강서경찰서,2018(상반기),남성,소음,6


In [351]:
# 폭력 건수만 모아보기
police_vio = police_vio[police_vio['inc_info'].str.contains('폭력')].groupby(['jur_stn','year']).count()
police_vio.reset_index(inplace=True)
police_vio.rename(columns={'month' : 'vio_cnt'},inplace = True)
police_vio.drop(columns=['report_sx','inc_info'],inplace = True)

# 2018(상반기) 5배
for i in range(len(police_vio)):
    if police_vio['year'][i] == '2018(상반기)':
        police_vio['vio_cnt'][i] *= 5
police_vio

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,jur_stn,year,vio_cnt
0,마산동부경찰서,2018(상반기),1305
1,마산동부경찰서,2018(하반기),1547
2,마산동부경찰서,2019(상반기),1751
3,마산동부경찰서,2019(하반기),1715
4,마산동부경찰서,2020(상반기),1511
...,...,...,...
238,창원중부경찰서,2018(하반기),1719
239,창원중부경찰서,2019(상반기),1883
240,창원중부경찰서,2019(하반기),1830
241,창원중부경찰서,2020(상반기),1550


In [352]:
# 여기도 세종 결측치 채우기
police_vio = pd.merge(base_df,police_vio,how='outer',on=('jur_stn','year'))
police_vio.fillna(0.0,inplace=True)
police_vio

,jur_stn,year,vio_cnt
0,서울중부경찰서,2018(상반기),1540.0
1,서울종로경찰서,2018(상반기),600.0
2,서울남대문경찰서,2018(상반기),1095.0
3,서울서대문경찰서,2018(상반기),2000.0
4,서울혜화경찰서,2018(상반기),1290.0
...,...,...,...
241,창원서부경찰서,2020(하반기),1282.0
242,마산중부경찰서,2020(하반기),1107.0
243,마산동부경찰서,2020(하반기),1430.0
244,진주경찰서,2020(하반기),2347.0


In [353]:
police_vio[police_vio.jur_stn.str.contains('세종')]

,jur_stn,year,vio_cnt
34,세종경찰서,2018(상반기),470.0
75,세종경찰서,2018(하반기),756.0
116,세종경찰서,2019(상반기),1361.0
154,세종경찰서,2019(하반기),0.0
198,세종경찰서,2020(상반기),0.0
239,세종경찰서,2020(하반기),0.0


## 3-3. 강도,살인 작업

In [354]:
police_mur_rob = df5.copy()
police_mur_rob.head(3)

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,6
1,서울송파경찰서,2018(상반기),여성,폭력,6
2,서울강서경찰서,2018(상반기),남성,소음,6


In [355]:
# 강도, 살인만 모아보기
police_mur_rob = df5[df5['inc_info'].str.contains('강도')|df5['inc_info'].str.contains('살인')].groupby(['jur_stn','year']).count()
police_mur_rob.reset_index(inplace=True)
police_mur_rob.rename(columns={'month' : 'mur_rob_cnt'},inplace = True)
police_mur_rob.drop(columns=['report_sx','inc_info'],inplace = True)

# 2018(상반기) 5배
for i in range(len(police_mur_rob)):
    if police_mur_rob['year'][i] == '2018(상반기)':
        police_mur_rob['mur_rob_cnt'][i] *= 5
police_mur_rob

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,jur_stn,year,mur_rob_cnt
0,마산동부경찰서,2018(하반기),2
1,마산동부경찰서,2019(상반기),1
2,마산동부경찰서,2019(하반기),8
3,마산동부경찰서,2020(하반기),4
4,마산중부경찰서,2018(하반기),8
...,...,...,...
177,창원중부경찰서,2018(하반기),2
178,창원중부경찰서,2019(상반기),1
179,창원중부경찰서,2019(하반기),1
180,창원중부경찰서,2020(상반기),2


In [356]:
# base 기준 결측치 채우기
police_mur_rob = pd.merge(base_df,police_mur_rob,how='outer',on=('jur_stn','year'))
police_mur_rob.fillna(0.0,inplace=True)
police_mur_rob

,jur_stn,year,mur_rob_cnt
0,서울중부경찰서,2018(상반기),10.0
1,서울종로경찰서,2018(상반기),0.0
2,서울남대문경찰서,2018(상반기),0.0
3,서울서대문경찰서,2018(상반기),0.0
4,서울혜화경찰서,2018(상반기),0.0
...,...,...,...
241,창원서부경찰서,2020(하반기),1.0
242,마산중부경찰서,2020(하반기),1.0
243,마산동부경찰서,2020(하반기),4.0
244,진주경찰서,2020(하반기),2.0


## 3-4. 교통사고

In [357]:
police_car = df5.copy()
police_car.head(3)

,jur_stn,year,report_sx,inc_info,month
0,서울서부경찰서,2018(상반기),남성,분실습득,6
1,서울송파경찰서,2018(상반기),여성,폭력,6
2,서울강서경찰서,2018(상반기),남성,소음,6


In [358]:
# 강도, 살인만 모아보기
police_car = df5[df5['inc_info'].str.contains('교통사고')].groupby(['jur_stn','year']).count()
police_car.reset_index(inplace=True)
police_car.rename(columns={'month' : 'car_cnt'},inplace = True)
police_car.drop(columns=['report_sx','inc_info'],inplace = True)

# 2018(상반기) 5배
for i in range(len(police_car)):
    if police_car['year'][i] == '2018(상반기)':
        police_car['car_cnt'][i] *= 5
police_car

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,jur_stn,year,car_cnt
0,마산동부경찰서,2018(상반기),1375
1,마산동부경찰서,2018(하반기),1752
2,마산동부경찰서,2019(상반기),2007
3,마산동부경찰서,2019(하반기),1910
4,마산동부경찰서,2020(상반기),1832
...,...,...,...
238,창원중부경찰서,2018(하반기),2334
239,창원중부경찰서,2019(상반기),2697
240,창원중부경찰서,2019(하반기),2819
241,창원중부경찰서,2020(상반기),2486


In [359]:
# base 기준 결측치 채우기
police_car = pd.merge(base_df,police_car,how='outer',on=('jur_stn','year'))
police_car.fillna(0.0,inplace=True)
police_car

,jur_stn,year,car_cnt
0,서울중부경찰서,2018(상반기),1500.0
1,서울종로경찰서,2018(상반기),760.0
2,서울남대문경찰서,2018(상반기),690.0
3,서울서대문경찰서,2018(상반기),1645.0
4,서울혜화경찰서,2018(상반기),880.0
...,...,...,...
241,창원서부경찰서,2020(하반기),1984.0
242,마산중부경찰서,2020(하반기),1301.0
243,마산동부경찰서,2020(하반기),1919.0
244,진주경찰서,2020(하반기),3169.0


# 4. 데이터 5번 통합본

In [360]:
# 성별, 폭력, 강도살인, 교통사고 다합치기

In [361]:
print(len(police_sx))
print(len(police_vio))
print(len(police_mur_rob))
print(len(police_car))
merge_list = [police_sx,police_vio,police_mur_rob,police_car]

246
246
246
246


In [374]:
# 각 데이터 착업한것 하나로 통일
temp = base_df
for i in merge_list:
    temp = pd.merge(temp,i, how='outer', on=('jur_stn','year'))

# 각 데이터 타입 int로 변경
columns_list=temp.columns[2:]
for i in columns_list:
    temp[i] = temp[i].astype(int)
    
df5_total = temp

In [375]:
df5_total

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt
0,서울중부경찰서,2018(상반기),13730,3430,6310,23470,1540,10,1500
1,서울종로경찰서,2018(상반기),8215,2260,4315,14790,600,0,760
2,서울남대문경찰서,2018(상반기),9315,1430,3380,14125,1095,0,690
3,서울서대문경찰서,2018(상반기),18685,9855,10670,39210,2000,0,1645
4,서울혜화경찰서,2018(상반기),11340,2370,5530,19240,1290,0,880
...,...,...,...,...,...,...,...,...,...
241,창원서부경찰서,2020(하반기),11915,2751,6897,21563,1282,1,1984
242,마산중부경찰서,2020(하반기),9913,2094,5405,17412,1107,1,1301
243,마산동부경찰서,2020(하반기),13687,3432,6906,24025,1430,4,1919
244,진주경찰서,2020(하반기),18826,4426,11375,34627,2347,2,3169


In [376]:
df5_total.to_csv('./5.112신고_total',index=False)

In [378]:
df5_total = pd.read_csv('./5.112신고_total')
df5_total

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt
0,서울중부경찰서,2018(상반기),13730,3430,6310,23470,1540,10,1500
1,서울종로경찰서,2018(상반기),8215,2260,4315,14790,600,0,760
2,서울남대문경찰서,2018(상반기),9315,1430,3380,14125,1095,0,690
3,서울서대문경찰서,2018(상반기),18685,9855,10670,39210,2000,0,1645
4,서울혜화경찰서,2018(상반기),11340,2370,5530,19240,1290,0,880
...,...,...,...,...,...,...,...,...,...
241,창원서부경찰서,2020(하반기),11915,2751,6897,21563,1282,1,1984
242,마산중부경찰서,2020(하반기),9913,2094,5405,17412,1107,1,1301
243,마산동부경찰서,2020(하반기),13687,3432,6906,24025,1430,4,1919
244,진주경찰서,2020(하반기),18826,4426,11375,34627,2347,2,3169


In [381]:
# 데이터 백업을 위한 로컬 저장소 활용
csv_download_link(df5_total,'5.112신고_total.csv')

/opt/app-root/src/5.112신고_total.csv

Press enter to delete the file after you have downloaded it.


# 피벗 참고

In [131]:
test = temp_data[(temp_data['jur_stn']==police_sx['jur_stn'].iloc[3])&(temp_data['year']==police_sx['year'].iloc[3])]

In [132]:
test

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성
1,마산동부경찰서,2018(하반기),0,0,0


In [133]:
test['남성'] = police_sx['report_sx_cnt'].iloc[0]

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [134]:
test

,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성,남성
1,마산동부경찰서,2018(하반기),0,0,0,2097


In [135]:
test.index

Int64Index([1], dtype='int64')

In [127]:
temp_data['report_sx_'+'남성'].iloc[test.index] = police_sx['report_sx_cnt'].iloc[0]
temp_data

/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,jur_stn,year,report_sx_남성,report_sx_불상,report_sx_여성
0,마산동부경찰서,2018(상반기),2097,0,0
1,마산동부경찰서,2018(하반기),0,0,0
2,마산동부경찰서,2019(상반기),0,0,0
3,마산동부경찰서,2019(하반기),0,0,0
4,마산동부경찰서,2020(상반기),0,0,0
...,...,...,...,...,...
279,창원중부경찰서,2019(상반기),0,0,0
280,창원중부경찰서,2019(하반기),0,0,0
281,창원중부경찰서,2020(상반기),0,0,0
282,창원중부경찰서,2020(하반기),0,0,0
